In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import re
import sklearn.metrics.pairwise as pw
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances

import warnings
warnings.filterwarnings('ignore')

In [2]:
# data import
df = pd.read_csv('./movielens_data/ratings.csv') 
movie_titles=pd.read_csv('./movielens_data/movies.csv')

---

In [3]:
# 상관 관계를 활용한 추천
def corr_recommender(df, film_name, min_num_reviews):
    
    df = pd.merge(df, movie_titles, on='movieId')
    ratings_df = pd.DataFrame(df.groupby('title')['rating'].mean())
    ratings_df.rename(columns={'rating': 'average_rating'}, inplace=True)
    ratings_df['num_of_ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
        
    user_movie_matrix = df.pivot_table(values='rating' , index='userId' , columns='title' )
    
    film_x_user_ratings = user_movie_matrix[film_name]
    similar_to_film_x = user_movie_matrix.corrwith(film_x_user_ratings)
    
    corr_film_x = pd.DataFrame(similar_to_film_x, columns=['Correlation'])
    corr_film_x.dropna(inplace=True)
    # Join ratings info to enbale filtering of films with low nums of ratings
    corr_film_x = corr_film_x.join(ratings_df['num_of_ratings'])
    # Apply filter
    new_corr_film_x = corr_film_x[corr_film_x['num_of_ratings'] >= min_num_reviews]
    # Sort intp ascending order
    return new_corr_film_x.sort_values('Correlation',ascending=False).iloc[1:10, :]

In [4]:
corr_recommender(df, 'Forrest Gump (1994)', 100)

,Correlation,num_of_ratings
title,,
Good Will Hunting (1997),0.484042,141
Aladdin (1992),0.464268,183
American History X (1998),0.457287,129
"Truman Show, The (1998)",0.432556,125
Braveheart (1995),0.416976,237
Ferris Bueller's Day Off (1986),0.405830,109
Mrs. Doubtfire (1993),0.401408,144
Full Metal Jacket (1987),0.397241,102
Saving Private Ryan (1998),0.390074,188
